In [1]:
from dask.distributed import Client, progress
client = Client(n_workers=4, threads_per_worker=3, memory_limit='2GB')
client

C:\Users\venka\anaconda3\lib\site-packages\distributed\node.py:160: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 59799 instead
  warnings.warn(


Connection method: Cluster object,Cluster type: distributed.LocalCluster
Dashboard: http://127.0.0.1:59799/status,
Dashboard: http://127.0.0.1:59799/status,Workers: 4
Total threads: 12,Total memory: 7.45 GiB
Status: running,Using processes: True
Comm: tcp://127.0.0.1:59800,Workers: 4
Dashboard: http://127.0.0.1:59799/status,Total threads: 12
Started: Just now,Total memory: 7.45 GiB
Comm: tcp://127.0.0.1:59827,Total threads: 3
Dashboard: http://127.0.0.1:59828/status,Memory: 1.86 GiB
Nanny: tcp://127.0.0.1:59806,


In [2]:
from pprint import pprint
from time import time
import logging

from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_extraction.text import TfidfTransformer
from sklearn.linear_model import SGDClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.pipeline import Pipeline

In [3]:
# Scale Up: set categories=None to use all the categories
categories = [
    'alt.atheism',
    'talk.religion.misc',
]

print("Loading 20 newsgroups dataset for categories:")
print(categories)

data = fetch_20newsgroups(subset='train', categories=categories)
print("%d documents" % len(data.filenames))
print("%d categories" % len(data.target_names))
print()


Loading 20 newsgroups dataset for categories:
['alt.atheism', 'talk.religion.misc']
857 documents
2 categories



In [6]:
pipeline = Pipeline([
    ('vect', HashingVectorizer()),
    ('tfidf', TfidfTransformer()),
    ('clf', SGDClassifier(max_iter=1000)),
])

In [7]:
parameters = {
    'tfidf__use_idf': (True, False),
    'tfidf__norm': ('l1', 'l2'),
    'clf__alpha': (0.00001, 0.000001),
    # 'clf__penalty': ('l2', 'elasticnet'),
    # 'clf__n_iter': (10, 50, 80),
}

In [8]:
grid_search = GridSearchCV(pipeline, parameters, n_jobs=-1, verbose=1, cv=3, refit=False)

In [9]:
import joblib

with joblib.parallel_backend('dask'):
    grid_search.fit(data.data, data.target)

Fitting 3 folds for each of 8 candidates, totalling 24 fits


In [10]:
import dask
import dask.array as da
from dask_ml.datasets import make_classification


n, d = 100000, 100

X, y = make_classification(n_samples=n, n_features=d,
                           chunks=n // 10, flip_y=0.2)
X

dask.array<normal, shape=(100000, 100), dtype=float64, chunksize=(10000, 100), chunktype=numpy.ndarray>

In [28]:
from dask_ml.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, y)
X_train

dask.array<concatenate, shape=(90000, 100), dtype=float64, chunksize=(9000, 100), chunktype=numpy.ndarray>

In [46]:
from sklearn.ensemble import VotingClassifier
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import LogisticRegression
from sklearn.svm import SVC

import sklearn.datasets

X, y = sklearn.datasets.make_classification(n_samples=1_000_000, n_features=20)

In [47]:
classifiers = [
    ('sgd', SGDClassifier(max_iter=1000)),
    ('logisticregression', LogisticRegression()),
    ('svc', SVC(gamma='auto')),
]
clf = VotingClassifier(classifiers, n_jobs=-1)

In [ ]:
%%time
with joblib.parallel_backend("dask"):
    clf.fit(X, y)

print(clf)

distributed.utils - ERROR - '<' not supported between instances of 'NoneType' and 'tuple'
Traceback (most recent call last):
  File "C:\Users\venka\anaconda3\lib\site-packages\distributed\utils.py", line 681, in log_errors
    yield
  File "C:\Users\venka\anaconda3\lib\site-packages\distributed\dashboard\components\scheduler.py", line 3391, in graph_doc
    graph = TaskGraph(scheduler, sizing_mode="stretch_both")
  File "C:\Users\venka\anaconda3\lib\site-packages\distributed\dashboard\components\scheduler.py", line 1939, in __init__
    self.layout = GraphLayout(scheduler)
  File "C:\Users\venka\anaconda3\lib\site-packages\distributed\diagnostics\graph_layout.py", line 39, in __init__
    self.update_graph(
  File "C:\Users\venka\anaconda3\lib\site-packages\distributed\diagnostics\graph_layout.py", line 49, in update_graph
    stack = sorted(tasks, key=lambda k: priority.get(k, 0), reverse=True)
TypeError: '<' not supported between instances of 'NoneType' and 'tuple'
tornado.applicatio

In [32]:
inc.score(X_test, y_test)

NotFittedError: This Incremental instance is not fitted yet. Call 'fit' with appropriate arguments before using this estimator.